In [1]:
import  pandas as pd

In [3]:
dataset=pd.read_table(r'C:\Users\lenovo\Downloads\SMSSpamCollection',names=['label','Messages'])

In [4]:
dataset.head()

,label,Messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
#converting labels from strings to 0's and 1's
dataset['label']=dataset.label.map({'ham':0,'spam':1})

In [6]:
print(dataset.shape)

(5572, 2)


In [7]:
dataset.head()

,label,Messages
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
# Converting text to lower case
lower_case_documents=[]
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)    

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


In [18]:
#removing punctuations
import string
sans_punctuation_document=[]

for i in lower_case_documents:
    sans_punctuation_document.append(i.translate(str.maketrans('', '', string.punctuation)))
print(sans_punctuation_document)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


In [20]:
#spliting words into list

preprocessed_document=[]
for i in sans_punctuation_document:
    preprocessed_document.append(i.split())
    
print(preprocessed_document)   

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


In [22]:
#count word frequencies

from collections import Counter
freq_list=[]
for i in preprocessed_document:
    freq_list.append(Counter(i))
print(freq_list)    

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}), Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}), Counter({'call': 1, 'me': 1, 'now': 1}), Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


In [24]:
#implenting the above three steps using scikit 
 
from sklearn.feature_extraction.text import CountVectorizer
count_vector=CountVectorizer()
print(count_vector)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [26]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

count_vector.fit(documents)
count_vector.get_feature_names()


['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [27]:
#converting the count_vector to an array

doc_array=count_vector.transform(documents).toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

In [29]:
#adding feature names to the matrix

freq_matrix=pd.DataFrame(doc_array,columns=count_vector.get_feature_names())
freq_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


In [30]:
#splittng dataset into train data and test data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(dataset["Messages"],dataset["label"],random_state=1)

print('No of rows in dataset',dataset.shape)
print('NO of rows in Train dataset',X_train.shape)
print('No of rows in Test Dataset',X_test.shape)


No of rows in dataset (5572, 2)
NO of rows in Train dataset (4179,)
No of rows in Test Dataset (1393,)


In [31]:
#tranform X_train and X_test

training_data=count_vector.fit_transform(X_train)
testing_data=count_vector.transform(X_test)

In [34]:
# train the model using MultinomialNBclassifier
from sklearn.naive_bayes import MultinomialNB

model=MultinomialNB()
model.fit(training_data,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
#make predictions

prediction=model.predict(testing_data)

In [42]:
from sklearn.metrics import accuracy_score,precision_score, recall_score ,f1_score

print('Accuracy:',accuracy_score(y_test,prediction))
print('Precision:',precision_score(y_test,prediction))
print('Recalll:',recall_score(y_test,prediction))
print('F1_score:',f1_score(y_test,prediction))

Accuracy: 0.9885139985642498
Precision: 0.9720670391061452
Recalll: 0.9405405405405406
F1_score: 0.9560439560439562
